In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


In [3]:
# read and saperate dataset
path = "C:/Users/shuji/Documents/projects/bank/bank-additional1.csv"
#df = pd.read_csv(path,header = None,delimiter=";")
df = pd.read_csv(path)
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,admin.,married,basic.6y,no,yes,yes,cellular,jul,thu,1,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,no
4115,39,admin.,married,high.school,no,yes,no,telephone,jul,fri,1,999,0,nonexistent,1.4,93.918,-42.7,4.959,5228.1,no
4116,27,student,single,high.school,no,no,no,cellular,may,mon,2,999,1,failure,-1.8,92.893,-46.2,1.354,5099.1,no
4117,58,admin.,married,high.school,no,no,no,cellular,aug,fri,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no


In [3]:
#Convert categorical variable into dummy/indicator variables.
#y and contact have two categories
df["y"]=df.y.apply(lambda x: 0 if x== "no" else 1)
df["contact"] = df.contact.apply(lambda x: 1 if x== "cellular" else 0)

In [ ]:
#购买和未购买的人的比例
df['y']=np.where(df['y'] =='0', 0, df['y'])
df['y']=np.where(df['y'] =='1', 1, df['y'])
df['y'].value_counts()

In [ ]:
#直方图
sns.countplot(x='y', data = df, palette='hls')
plt.show()
plt.savefig('count_plot')

In [4]:
#用均值查看一下0与1的情况
df.groupby('y').mean()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
y,,,,,,,,,
no,39.895311,2.605780,982.763086,0.141767,0.240185,93.599677,-40.586723,3.802826,5175.502072
yes,41.889135,1.980044,778.722838,0.585366,-1.177384,93.417268,-39.786475,2.145448,5093.118625


In [ ]:
#check categories of categorical variable
# df.job.unique()
# df.marital.unique()
# df.education.unique()
# df.marital.unique()
# df.contact.unique()


In [ ]:
#计算其他特征值（如教育和婚姻状况）的分布
df.groupby('job').mean() 
df.groupby('marital').mean() 
df.groupby('education').mean()

In [ ]:
#我们发现具有不同job的人购买存款的频率不一样。 因此，job可以是良好的预测因素。
%matplotlib inline
table=pd.crosstab(df.job,df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Job title vs Purchase')
plt.xlabel('Job')
plt.ylabel('Proportion of Purchase')
plt.savefig('purchase_vs_job')

In [ ]:
#婚姻状况似乎不是好的预测因素
table=pd.crosstab(df.marital,df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Marital Status vs Purchase')
plt.xlabel('Marital Status')
plt.ylabel('Proportion of Customers')
plt.savefig('mariral_vs_pur_stack')

In [ ]:
#下面是教育属性
#可以看出教育似乎是结果变量的良好预测指标
table=pd.crosstab(df.education,df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Education vs Purchase')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')
plt.savefig('edu_vs_pur_stack')



In [ ]:
#下面是时间特征,一周工作时间不是预测结果的良好预测因素
table=pd.crosstab(df.day_of_week,df.y)#.plot(kind='bar')
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Day of Week vs Purchase')
plt.xlabel('Day of Week')
plt.ylabel('Proportion of Purchase')
plt.savefig('dow_vs_purchase')

In [ ]:
#month与y的关系
pd.crosstab(df.month,df.y).plot(kind='bar')
plt.title('Purchase Frequency for Month')
plt.xlabel('Month')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_month_bar')
plt.show()


In [4]:
# count each categories of columns
def count_category(cate):
    dict = {}
    entries = np.unique(cate)
    for i in entries:
        dict[i] = len(cate[cate ==i])
    return dict

In [5]:
# get mode of column job
# other categorical variable have multiple categories,we repleace the unknown with mode of the column 
count_job = count_category(df["job"])
{a:b for a,b in sorted(count_job.items(), key = lambda item: item[1],reverse=True)}
df['job'].replace("unknown","admin.",inplace = True)
job_dum = pd.get_dummies(df.job,prefix="job")
df = pd.concat([df,job_dum],axis=1)

In [6]:
#marital
count_marital = count_category(df["marital"])
{a:b for a,b in sorted(count_marital.items(), key = lambda item: item[1],reverse=True)}
df['marital'].replace("unknown","married",inplace = True)
mari_dum = pd.get_dummies(df.marital,prefix="mari")
df = pd.concat([df,mari_dum],axis=1)

In [7]:
def education_to_num(a):
    edu = ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'professional.course', 'university.degree']
    list = [0, 0.15, 0.3, 0.45, 0.6, 0.75, 1]
    for i in range(7):
        if (a == edu[i]):
            return list[i]

In [8]:
#education have different levels
count_education = count_category(df["education"])
{a:b for a,b in sorted(count_education.items(), key = lambda item: item[1],reverse=True)}
df['education'].replace("unknown","university.degree",inplace = True)
df['education'] = df.education.apply(education_to_num)

In [9]:
#default: has credit in default?
count_dafault = count_category(df["default"])
default_dummies = pd.get_dummies(df.default)
df["default"] = df.default.apply(lambda x: 1 if x == "unknown" else 0)


In [10]:
#housing: has housing?
count_loan = count_category(df["loan"])
{a:b for a,b in sorted(count_loan.items(), key = lambda item: item[1],reverse=True)}
df["housing"].replace("unknown", "yes", inplace=True)
df["housing"] = df.housing.apply(lambda x: 1 if x == "yes" else 0)

In [11]:
# 'loan'
df["loan"].replace("unknown", "no", inplace=True)
df["loan"] = df.loan.apply(lambda x: 1 if x == "yes" else 0)

In [12]:
#month
month_dummies = pd.get_dummies(df.month)
df = pd.concat([df, month_dummies], axis=1)

In [13]:
#weekday
weekday_dummies = pd.get_dummies(df.day_of_week)
df = pd.concat([df, weekday_dummies], axis=1)
#train.drop(['day_of_week'], axis=1, inplace=True)

In [14]:
col = df.columns
col

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student', 'job_technician',
       'job_unemployed', 'mari_divorced', 'mari_married', 'mari_single', 'apr',
       'aug', 'dec', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri',
       'mon', 'thu', 'tue', 'wed'],
      dtype='object')

In [15]:
df.drop(['marital'], axis=1, inplace=True)
df.drop(['job'], axis=1, inplace=True)
df.drop(['month'], axis=1, inplace=True)
df.drop(['day_of_week'], axis=1, inplace=True)
df.drop(['poutcome'], axis=1, inplace=True)

In [16]:
df

,age,education,default,housing,loan,contact,campaign,pdays,previous,emp.var.rate,...,mar,may,nov,oct,sep,fri,mon,thu,tue,wed
0,30,0.45,0,1,0,1,2,999,0,-1.8,...,0,1,0,0,0,1,0,0,0,0
1,39,0.60,0,0,0,0,4,999,0,1.1,...,0,1,0,0,0,1,0,0,0,0
2,25,0.60,0,1,0,0,1,999,0,1.4,...,0,0,0,0,0,0,0,0,0,1
3,38,0.45,0,1,0,0,3,999,0,1.4,...,0,0,0,0,0,1,0,0,0,0
4,47,1.00,0,1,0,1,1,999,0,-0.1,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,30,0.30,0,1,1,1,1,999,0,1.4,...,0,0,0,0,0,0,0,1,0,0
4115,39,0.60,0,1,0,0,1,999,0,1.4,...,0,0,0,0,0,1,0,0,0,0
4116,27,0.60,0,0,0,1,2,999,1,-1.8,...,0,1,0,0,0,0,1,0,0,0
4117,58,0.60,0,0,0,1,1,999,0,1.4,...,0,0,0,0,0,1,0,0,0,0


In [17]:
# divided label
label = df["y"]
df.drop(["y"], axis=1, inplace=True)

In [55]:
# split train & test data   X_test & y_test are used for test case
#_X_train, X_test, _y_train, y_test are tuple and can not be changed 
col = df.columns
_X_train, X_test, _y_train, y_test  = train_test_split(df, label, test_size=0.2, random_state=42)
_X_train = np.array(_X_train)
X_test = np.array(X_test)
_y_train = np.array(_y_train)
y_test = np.array(y_test)

In [56]:
neg_X_train = _X_train[_y_train ==0]
neg_y_train = _y_train[_y_train ==0]

In [57]:
print(len(neg_X_train))

2936


In [58]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy= 1/3,random_state = 42)
X_train,y_train = sm.fit_sample(_X_train,_y_train)

In [59]:
print(len(X_train))

3914


In [60]:
print(len(_X_train))

3295


In [61]:
# normalize
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [62]:
# -------------------feature-selection--------------------
# Method1: SelectKBest,k 为训练后变成35个特征
select = SelectKBest(chi2, k=35)
select.fit(X_train, y_train)
#print(select.pvalues_ * 10000)
select.get_support()

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True, False,  True, False, False,  True])

In [48]:
# Method2: RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
forest.fit(X_train, y_train)
name_list = df.columns
importance = forest.feature_importances_
imp_result = np.argsort(importance)[::-1]
imp_result [0:5]
for i in range(X_train.shape[1]):
    print("%2d. %-*s %f"%(i+1, 30, name_list[i], importance[imp_result[i]]) )

 1. age                            0.108844
 2. education                      0.083084
 3. default                        0.080347
 4. housing                        0.056690
 5. loan                           0.053183
 6. contact                        0.049878
 7. campaign                       0.044083
 8. pdays                          0.038271
 9. previous                       0.036156
10. emp.var.rate                   0.035260
11. cons.price.idx                 0.033171
12. cons.conf.idx                  0.032739
13. euribor3m                      0.030616
14. nr.employed                    0.023624
15. job_admin.                     0.022036
16. job_blue-collar                0.021818
17. job_entrepreneur               0.021293
18. job_housemaid                  0.020964
19. job_management                 0.020087
20. job_retired                    0.018820
21. job_self-employed              0.018524
22. job_services                   0.015971
23. job_student                 

In [49]:
# reduce features
delete_index = [i for i in range(len(importance)) if importance[i] < 0.004]
X_train = np.delete(X_train, delete_index, axis=1)
X_test = np.delete(X_test, delete_index, axis=1)

In [73]:
#-------------------------------K-fold----------------------------------
#StratifiedKFold用法类似 Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
skf = StratifiedKFold(n_splits=5, shuffle=True)

index_train = []
index_test = []

for train_index, test_index in skf.split(X_train, y_train):
    index_train.append(train_index)
    index_test.append(test_index)

In [81]:
#Accuracy
from sklearn.metrics import accuracy_score
def getAccuracy(a, y_test):
    a[a >= 0.5] = 1
    a[a < 0.5] = 0
    return accuracy_score(y_test, a)

In [ ]:
# ROC plot
def plotROC(y_test, y_score):
    # Compute ROC curve and ROC area for each class

    fpr, tpr, threshold = roc_curve(y_test, y_score, pos_label=1)  ###计算真正率和假正率
    roc_auc = auc(fpr, tpr)  ###计算auc的值

    plt.figure()
    lw = 2
    plt.figure(figsize=(10, 10))
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)  ###假正率为横坐标，真正率为纵坐标做曲线
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
#Confusion matrix
def plotConfusion(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm_normalized)
    plt.imshow(cm_normalized)
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

In [75]:
from sklearn.ensemble import RandomForestRegressor
def RF_gridSearch(Xtrain, ytrain, index_train, index_test):
    n_estimators = [100, 200, 300]
    max_depth = [5, 10, 15, 20]
    min_samples_split = [2, 3, 4]
    res = np.zeros((len(n_estimators),len(max_depth),len(min_samples_split)))
    f1 = np.zeros((len(n_estimators), len(max_depth), len(min_samples_split)))

    for index in range(len(index_train)):
        X_train = Xtrain[index_train[index], :]
        y_train = ytrain[index_train[index]]
        X_test = Xtrain[index_test[index], :]
        y_test = ytrain[index_test[index]]
        pos_X_train = X_train[y_train == 1]
        pos_y_train = y_train[y_train == 1]
        X_train = np.vstack((np.vstack((np.vstack((X_train, pos_X_train)), pos_X_train)), pos_X_train))
        y_train = np.hstack((np.hstack((np.hstack((y_train, pos_y_train)), pos_y_train)), pos_y_train))

        for i in range(len(n_estimators)):
            for j in range(len(max_depth)):
                for k in range(len(min_samples_split)):
                    forest = RandomForestRegressor(n_estimators=n_estimators[i],
                                                   random_state=0,
                                                   oob_score=True,
                                                   max_depth=max_depth[j],
                                                   min_samples_split=min_samples_split[k],
                                                   n_jobs=-1)
                    forest.fit(X_train, y_train)
                    output = forest.predict(X_test)
                    fpr, tpr, thresholds = metrics.roc_curve(y_test, output, pos_label=1)
                    res[i][j][k] += metrics.auc(fpr, tpr)
                    output[output >= 0.5] = 1
                    output[output < 0.5] = 0
                    f1[i][j][k] += f1_score(y_test, output, average='binary')
    return res / 5, f1 / 5


In [76]:
gridSearchResult, f1 = RF_gridSearch(X_train, y_train, index_train, index_test)

In [77]:
gridSearchResult

array([[[0.88568137, 0.88565878, 0.88567963],
        [0.91619774, 0.91603664, 0.91627719],
        [0.91765273, 0.91821774, 0.91801583],
        [0.91663148, 0.91742437, 0.91748888]],

       [[0.88579621, 0.88578578, 0.88585368],
        [0.91576722, 0.91578282, 0.91600703],
        [0.91843001, 0.91842292, 0.91861815],
        [0.91730589, 0.91754825, 0.91807377]],

       [[0.88657847, 0.88657151, 0.8866203 ],
        [0.91676907, 0.91672363, 0.91695089],
        [0.91895037, 0.91863065, 0.91906605],
        [0.91776164, 0.91834698, 0.91834229]]])

In [82]:
#通过gridSearch得到最优参数，放到rf里面
rf_clf = RandomForestRegressor(n_estimators=200,
                                random_state=0,
                                oob_score=True,
                                max_depth=5,
                                n_jobs=-1,
                                min_samples_split=2)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, rf_pred, pos_label=1)
#mf.plotROC(y_test, rf_pred)
print("Random Forest AUC: ",metrics.auc(fpr, tpr))
print("Random Forest Accuracy: ", getAccuracy(rf_pred, y_test))
print("Random Forest F Score: ", f1_score(y_test, rf_pred, average='binary'))
#mf.plotConfusion(y_test, rf_pred)


Random Forest AUC:  0.7555016037063436
Random Forest Accuracy:  0.8944174757281553
Random Forest F Score:  0.423841059602649


In [83]:
#--------------------SVM-------------------------

# gridSearchResult = mf.SVM_gridSearch(X_train, y_train, index_train, index_test)
def SVM_gridSearch(Xtrain, ytrain, index_train, index_test):
    C = [1, 3, 5, 10, 15, 20]
    gamma = [0.001, 0.005, 0.1, 0.5, 1, 2, 5]
    res = np.zeros((len(C),len(gamma)))
    f1 = np.zeros((len(C), len(gamma)))

    for index in range(len(index_train)):
        X_train = Xtrain[index_train[index], :]
        y_train = ytrain[index_train[index]]
        X_test = Xtrain[index_test[index], :]
        y_test = ytrain[index_test[index]]
        pos_X_train = X_train[y_train == 1]
        pos_y_train = y_train[y_train == 1]
        X_train = np.vstack((np.vstack((np.vstack((X_train, pos_X_train)), pos_X_train)), pos_X_train))
        y_train = np.hstack((np.hstack((np.hstack((y_train, pos_y_train)), pos_y_train)), pos_y_train))

        for i in range(len(C)):
            for j in range(len(gamma)):
                    clf_svm = svm.SVR(C=C[i],
                                      gamma=gamma[j])
                    clf_svm.fit(X_train, y_train)
                    output = clf_svm.predict(X_test)
                    res[i][j] += roc_auc_score(y_test, output)
                    output[output >= 0.5] = 1
                    output[output < 0.5] = 0
                    f1[i] += f1_score(y_test, output, average='binary')
    return res / 5, f1



SVM AUC:  0.7738625564267047
SVM Accuracy:  0.8822815533980582
SVM F Score:  0.4756756756756757


In [ ]:
svm_clf = svm.SVR(C=6,
                  gamma=0.002)
svm_clf.fit(X_train, y_train)
svm_pred = svm_clf.predict(X_test)
fpr2, tpr2, thresholds2 = metrics.roc_curve(y_test, svm_pred, pos_label=1)
#mf.plotROC(y_test, svm_pred)
print("SVM AUC: ",metrics.auc(fpr2, tpr2))
print("SVM Accuracy: ", getAccuracy(svm_pred, y_test))
print("SVM F Score: ", f1_score(y_test, svm_pred, average='binary'))
#mf.plotConfusion(y_test, svm_pred)

In [84]:
#--------------------Naive Bayes-------------------------
#正态分布
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
gnb_pred = gnb_clf.predict_proba(X_test)[:, 1]
fpr3, tpr3, thresholds3 = metrics.roc_curve(y_test, gnb_pred, pos_label=1)
#mf.plotROC(y_test, gnb_pred)
print("Naive Bayes AUC: ",metrics.auc(fpr3, tpr3))
print("Naive Bayes Accuracy: ", getAccuracy(gnb_pred, y_test))
print("Naive Bayes F Score: ", f1_score(y_test, gnb_pred, average='binary'))
#mf.plotConfusion(y_test, gnb_pred)

Naive Bayes AUC:  0.7408455096222382
Naive Bayes Accuracy:  0.8543689320388349
Naive Bayes F Score:  0.4495412844036697


In [86]:
#--------------------KNN-------------------------

# gridSearchResult, f1Score = mf.KNN_gridSearch(X_train, y_train, index_train, index_test)

knn_clf = KNeighborsClassifier(n_neighbors=20)
knn_clf.fit(X_train, y_train)
knn_pred = knn_clf.predict_proba(X_test)[:, 1]
fpr4, tpr4, thresholds4 = metrics.roc_curve(y_test, knn_pred, pos_label=1)
#plotROC(y_test, knn_pred)
print("KNN AUC: ", metrics.auc(fpr4, tpr4))
print("KNN Accuracy: ", getAccuracy(knn_pred, y_test))
print("KNN F Score: ", f1_score(y_test, knn_pred, average='binary'))
#plotConfusion(y_test, knn_pred)

KNN AUC:  0.7460130078403422
KNN Accuracy:  0.8555825242718447
KNN F Score:  0.4413145539906103


In [87]:
#---------------------xgboost-----------------------

data = xgb.DMatrix(X_train,label=y_train)
params={'booster':'gbtree',        # booster:{gbtree, gblinear}
	    'objective': 'binary:logistic',
	    'eval_metric':'auc',
	    'gamma':0.1,                # minimum loss reduction required to make a split
	    'max_depth':10,              # maximum tree depth
	    'lambda':1,                # L2 regular term in the object function
	    'subsample':0.7,
	    'colsample_bytree':0.7,     # rate of random choose columns
	    'colsample_bylevel':0.7,
	    'eta': 0.01,                # shrink parameter
	    'tree_method':'exact',      # 使用精确节点分裂的方法
	    'seed':0
	    }
watchlist = [(data,'train')]
model = xgb.train(params,dtrain=data,num_boost_round=500,evals=watchlist)
xg_test = xgb.DMatrix(X_test)
xgboost_pred = model.predict(xg_test)
fpr5, tpr5, thresholds5 = metrics.roc_curve(y_test, xgboost_pred, pos_label=1)
print("Xgboost AUC: ", metrics.auc(fpr5, tpr5))
print("Xgboost Accuracy: ", getAccuracy(xgboost_pred, y_test))
print("Xgboost F Score: ", f1_score(y_test, xgboost_pred, average='binary'))


[0]	train-auc:0.89007
[1]	train-auc:0.92327
[2]	train-auc:0.93137
[3]	train-auc:0.93674
[4]	train-auc:0.94142
[5]	train-auc:0.94433
[6]	train-auc:0.94729
[7]	train-auc:0.94970
[8]	train-auc:0.94973
[9]	train-auc:0.95006
[10]	train-auc:0.94986
[11]	train-auc:0.94954
[12]	train-auc:0.95144
[13]	train-auc:0.95153
[14]	train-auc:0.95170
[15]	train-auc:0.95280
[16]	train-auc:0.95304
[17]	train-auc:0.95434
[18]	train-auc:0.95493
[19]	train-auc:0.95548
[20]	train-auc:0.95591
[21]	train-auc:0.95631
[22]	train-auc:0.95614
[23]	train-auc:0.95591
[24]	train-auc:0.95586
[25]	train-auc:0.95591
[26]	train-auc:0.95578
[27]	train-auc:0.95562
[28]	train-auc:0.95592
[29]	train-auc:0.95610
[30]	train-auc:0.95680
[31]	train-auc:0.95684
[32]	train-auc:0.95719
[33]	train-auc:0.95759
[34]	train-auc:0.95742
[35]	train-auc:0.95804
[36]	train-auc:0.95800
[37]	train-auc:0.95808
[38]	train-auc:0.95812
[39]	train-auc:0.95861
[40]	train-auc:0.95901
[41]	train-auc:0.95907
[42]	train-auc:0.95937
[43]	train-auc:0.9600

[346]	train-auc:0.98909
[347]	train-auc:0.98917
[348]	train-auc:0.98922
[349]	train-auc:0.98930
[350]	train-auc:0.98932
[351]	train-auc:0.98934
[352]	train-auc:0.98940
[353]	train-auc:0.98946
[354]	train-auc:0.98951
[355]	train-auc:0.98955
[356]	train-auc:0.98959
[357]	train-auc:0.98963
[358]	train-auc:0.98965
[359]	train-auc:0.98970
[360]	train-auc:0.98974
[361]	train-auc:0.98979
[362]	train-auc:0.98983
[363]	train-auc:0.98984
[364]	train-auc:0.98988
[365]	train-auc:0.98994
[366]	train-auc:0.98996
[367]	train-auc:0.98999
[368]	train-auc:0.99001
[369]	train-auc:0.99007
[370]	train-auc:0.99011
[371]	train-auc:0.99016
[372]	train-auc:0.99020
[373]	train-auc:0.99024
[374]	train-auc:0.99026
[375]	train-auc:0.99027
[376]	train-auc:0.99029
[377]	train-auc:0.99033
[378]	train-auc:0.99038
[379]	train-auc:0.99039
[380]	train-auc:0.99042
[381]	train-auc:0.99046
[382]	train-auc:0.99050
[383]	train-auc:0.99055
[384]	train-auc:0.99057
[385]	train-auc:0.99063
[386]	train-auc:0.99069
[387]	train-auc: